# Sistema generador de transacciones fraudulentas

### Librerías varias

In [34]:
import os

import numpy as np
import pandas as pd

import datetime
import time

import random

%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid', {'axes.facecolor': '0.9'})

### Generación de clientes

In [35]:
def generate_customer_profiles_table(n_customers, random_state=0):
    
    np.random.seed(random_state)
        
    customer_id_properties=[]
    
    # Generate customer properties from random distributions 
    for cliente_id in range(n_customers):
        
        x_pos_cliente = np.random.uniform(0,100)
        y_pos_cliente = np.random.uniform(0,100)
        
        monto_promedio = np.random.uniform(5000,100000) # Arbitrary (but sensible) value 
        desviacion_promedio = monto_promedio/2 # Arbitrary (but sensible) value
        
        promedio_tx_por_dia = np.random.uniform(0,4) # Arbitrary (but sensible) value 
        
        customer_id_properties.append([cliente_id,
                                      x_pos_cliente, y_pos_cliente,
                                      monto_promedio, desviacion_promedio,
                                      promedio_tx_por_dia])
        
    customer_profiles_table = pd.DataFrame(customer_id_properties, columns=['CLIENTE_ID',
                                                                      'x_pos_cliente', 'y_pos_cliente',
                                                                      'monto_promedio', 'desviacion_promedio',
                                                                      'promedio_tx_por_dia'])
    
    return customer_profiles_table

### Generación de terminales

In [36]:
def generate_terminal_profiles_table(n_terminals, random_state=0):
    np.random.seed(random_state)
    
    terminal_id_properties = []
    group_locations = []
    ################
    ###### RE-ESCRIBIR CODE UBICACIONES #####################################################
    #hola
    # Define el número de grupos y la cantidad mínima y máxima de posiciones por grupo
    n_groups = np.random.randint(2, 6)  # Número aleatorio de grupos
    min_positions_per_group = 2
    max_positions_per_group = int(n_terminals / n_groups) + 1
    
    # Genera las ubicaciones para cada grupo
    for group in range(n_groups):
        n_positions = np.random.randint(min_positions_per_group, max_positions_per_group + 1)
        locations = np.random.uniform(0,50, size=(n_positions, 2))
        group_locations.append(locations)
    
    # Asigna las ubicaciones a los terminales
    for terminal_id in range(n_terminals):
        group_index = np.random.randint(n_groups)
        locations = group_locations[group_index]
        x_pos_terminal, y_pos_terminal = locations[np.random.randint(len(locations))]
        
        terminal_id_properties.append([terminal_id, x_pos_terminal, y_pos_terminal])

    print('Número de grupos: {}'.format(n_groups))
    print('Número de terminales: {}'.format(n_terminals))
    print('Número de terminales por grupo: {}'.format(max_positions_per_group))
    print('Ubicaciones de los grupos: {}'.format(group_locations))
    
    terminal_profiles_table = pd.DataFrame(terminal_id_properties, columns=['TERMINAL_ID', 'x_pos_terminal', 'y_pos_terminal'])
    
    return terminal_profiles_table

In [37]:
def get_list_terminals_within_radius(customer_profile, x_y_terminals, r):
    
    # Use numpy arrays in the following to speed up computations
    
    # Location (x,y) of customer as numpy array
    x_y_customer = customer_profile[['x_pos_cliente','y_pos_cliente']].values.astype(float)
    
    # Squared difference in coordinates between customer and terminal locations
    squared_diff_x_y = np.square(x_y_customer - x_y_terminals)
    
    # Sum along rows and compute suared root to get distance
    dist_x_y = np.sqrt(np.sum(squared_diff_x_y, axis=1))
    
    # Get the indices of terminals which are at a distance less than r
    terminales_disponibles = list(np.where(dist_x_y<r)[0])
    
    # Return the list of terminal IDs
    return terminales_disponibles

In [38]:
def extract_datetime_features(tx_datetime):
    features = {}
    datetime = pd.to_datetime(tx_datetime)
    features['year'] = datetime.year
    features['month'] = datetime.month
    features['day'] = datetime.day
    features['hour'] = datetime.hour
    features['minute'] = datetime.minute
    return features


### Generación de transacciones

In [39]:
def generate_transactions_table(customer_profile, start_date = "2023-01-01", nb_days = 10):
    
    customer_transactions = []
    
    random.seed(int(customer_profile.CLIENTE_ID))
    np.random.seed(int(customer_profile.CLIENTE_ID))
    
    # For all days
    for day in range(nb_days):
        
        # Random number of transactions for that day 
        nb_tx = np.random.poisson(customer_profile.promedio_tx_por_dia)
        
        # If nb_tx positive, let us generate transactions
        if nb_tx>0:
            
            for tx in range(nb_tx):
                
                # Time of transaction: Around noon, std 20000 seconds. This choice aims at simulating the fact that 
                # most transactions occur during the day.
                time_tx = int(np.random.normal(86400/2, 20000))
                
                # If transaction time between 0 and 86400, let us keep it, otherwise, let us discard it
                if (time_tx>0) and (time_tx<86400):
                    
                    # Amount is drawn from a normal distribution  
                    amount = np.random.normal(customer_profile.monto_promedio, customer_profile.desviacion_promedio)
                    
                    # If amount negative, draw from a uniform distribution
                    if amount<0:
                        amount = np.random.uniform(0,customer_profile.monto_promedio*2)
                    
                    amount=np.round(amount,decimals=2)
                    
                    if len(customer_profile.terminales_disponibles)>0:
                        
                        terminal_id = random.choice(customer_profile.terminales_disponibles)
                    
                        customer_transactions.append([time_tx+day*86400, day,
                                                      customer_profile.CLIENTE_ID, 
                                                      terminal_id, amount])
            
    customer_transactions = pd.DataFrame(customer_transactions, columns=['TIEMPO_DESDE_ULTIMA_TX_S', 'DIA_TX', 'CLIENTE_ID', 'TERMINAL_ID', 'MONTO_TX'])
    
    if len(customer_transactions)>0:
      customer_transactions['TX_DATETIME'] = pd.to_datetime(customer_transactions["TIEMPO_DESDE_ULTIMA_TX_S"], unit='s', origin=start_date)
      customer_transactions['datetime_features'] = customer_transactions['TX_DATETIME'].apply(extract_datetime_features)
      customer_transactions = pd.concat([customer_transactions.drop('datetime_features', axis=1), customer_transactions['datetime_features'].apply(pd.Series)], axis=1)
      customer_transactions = customer_transactions[['TX_DATETIME', 'CLIENTE_ID', 'TERMINAL_ID', 'MONTO_TX', 'TIEMPO_DESDE_ULTIMA_TX_S', 'DIA_TX', 'year', 'month', 'day', 'hour', 'minute']]

    
    return customer_transactions 

### Generación del dataset

In [40]:
def generate_dataset(n_customers = 10000, n_terminals = 1000000, nb_days=90, start_date="2023-01-01", r=5):
    
    start_time=time.time()
    customer_profiles_table = generate_customer_profiles_table(n_customers, random_state = 0)
    print("Generación tabla perfiles de cliente: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    terminal_profiles_table = generate_terminal_profiles_table(n_terminals, random_state = 1)
    print("Generación tabla perfiles de terminal: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    x_y_terminals = terminal_profiles_table[['x_pos_terminal','y_pos_terminal']].values.astype(float)
    customer_profiles_table['terminales_disponibles'] = customer_profiles_table.apply(lambda x : get_list_terminals_within_radius(x, x_y_terminals=x_y_terminals, r=r), axis=1)
    # With Pandarallel
    #customer_profiles_table['available_terminals'] = customer_profiles_table.parallel_apply(lambda x : get_list_closest_terminals(x, x_y_terminals=x_y_terminals, r=r), axis=1)
    customer_profiles_table['numero_terminales']=customer_profiles_table.terminales_disponibles.apply(len)
    print("Asociación cliente a terminales: {0:.2}s".format(time.time()-start_time))
    
    start_time=time.time()
    transactions_df=customer_profiles_table.groupby('CLIENTE_ID').apply(lambda x : generate_transactions_table(x.iloc[0], nb_days=nb_days)).reset_index(drop=True)
    # With Pandarallel
    #transactions_df=customer_profiles_table.groupby('CUSTOMER_ID').parallel_apply(lambda x : generate_transactions_table(x.iloc[0], nb_days=nb_days)).reset_index(drop=True)
    print("Generación de transacciones: {0:.2}s".format(time.time()-start_time))
    
    # Sort transactions chronologically
    transactions_df=transactions_df.sort_values('TX_DATETIME')
    # Reset indices, starting from 0
    transactions_df.reset_index(inplace=True,drop=True)
    transactions_df.reset_index(inplace=True)
    # TRANSACTION_ID are the dataframe indices, starting from 0
    transactions_df.rename(columns = {'index':'TRANSACTION_ID'}, inplace = True)

    #SE AÑADEN LAS COORDENADAS DEL TERMINAL PARA USARLAS EN EL ENTRENAMIENTO
    transactions_df = pd.merge(transactions_df, terminal_profiles_table[['TERMINAL_ID', 'x_pos_terminal', 'y_pos_terminal']], on='TERMINAL_ID', how='left')
    
    return (customer_profiles_table, terminal_profiles_table, transactions_df)

### Inserción de fraudes

In [41]:
def add_frauds(customer_profiles_table, terminal_profiles_table, transactions_df):
    
    # By default, all transactions are genuine
    transactions_df['TX_FRAUD']=0
    transactions_df['TX_FRAUD_SCENARIO']=0
    
        # Scenario 1
    transactions_df.loc[transactions_df.MONTO_TX>80000, 'TX_FRAUD']=1
    transactions_df.loc[transactions_df.MONTO_TX>80000, 'TX_FRAUD_SCENARIO']=1
    nb_frauds_scenario_1=transactions_df.TX_FRAUD.sum()
    print("Number of frauds from scenario 1: "+str(nb_frauds_scenario_1))
    
    # Scenario 2
    for day in range(transactions_df.DIA_TX.max()):
        
        compromised_terminals = terminal_profiles_table.TERMINAL_ID.sample(n=2, random_state=day)
        
        compromised_transactions=transactions_df[(transactions_df.DIA_TX>=day) & 
                                                    (transactions_df.DIA_TX<day+28) & 
                                                    (transactions_df.TERMINAL_ID.isin(compromised_terminals))]
                            
        transactions_df.loc[compromised_transactions.index,'TX_FRAUD']=1
        transactions_df.loc[compromised_transactions.index,'TX_FRAUD_SCENARIO']=2
    
    nb_frauds_scenario_2=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_1
    print("Number of frauds from scenario 2: "+str(nb_frauds_scenario_2))
    
    # Scenario 3
    for day in range(transactions_df.DIA_TX.max()):
        
        compromised_customers = customer_profiles_table.CLIENTE_ID.sample(n=3, random_state=day).values
        
        compromised_transactions=transactions_df[(transactions_df.DIA_TX>=day) & 
                                                    (transactions_df.DIA_TX<day+14) & 
                                                    (transactions_df.CLIENTE_ID.isin(compromised_customers))]
        
        nb_compromised_transactions=len(compromised_transactions)
        
        
        random.seed(day)
        index_fauds = random.sample(list(compromised_transactions.index.values),k=int(nb_compromised_transactions/3))
        
        transactions_df.loc[index_fauds,'MONTO_TX']=transactions_df.loc[index_fauds,'MONTO_TX']*5
        transactions_df.loc[index_fauds,'TX_FRAUD']=1
        transactions_df.loc[index_fauds,'TX_FRAUD_SCENARIO']=3
        
                             
    nb_frauds_scenario_3=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_2-nb_frauds_scenario_1
    print("Number of frauds from scenario 3: "+str(nb_frauds_scenario_3))
    
    #Scenario 4
    transactions_df.loc[transactions_df.hour>21, 'TX_FRAUD']=1
    transactions_df.loc[transactions_df.hour>21, 'TX_FRAUD_SCENARIO']=1
    nb_frauds_scenario_4=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_3-nb_frauds_scenario_2-nb_frauds_scenario_1
    print("Number of frauds from scenario 4: "+str(nb_frauds_scenario_4))
    return transactions_df

In [42]:
def get_stats(transactions_df):
    #Number of transactions per day
    nb_tx_per_day=transactions_df.groupby(['DIA_TX'])['CLIENTE_ID'].count()
    #Number of fraudulent transactions per day
    nb_fraud_per_day=transactions_df.groupby(['DIA_TX'])['TX_FRAUD'].sum()
    #Number of fraudulent cards per day
    nb_fraudcard_per_day=transactions_df[transactions_df['TX_FRAUD']>0].groupby(['DIA_TX']).CLIENTE_ID.nunique()
    
    return (nb_tx_per_day,nb_fraud_per_day,nb_fraudcard_per_day)

In [43]:
#CLIENTES
n_customers = 1000
customer_profiles_table = generate_customer_profiles_table(n_customers, random_state = 0)
#TERMINALES
n_terminals = 100
terminal_profiles_table = generate_terminal_profiles_table(n_terminals, random_state = 0)

# We first get the geographical locations of all terminals as a numpy array
x_y_terminals = terminal_profiles_table[['x_pos_terminal','y_pos_terminal']].values.astype(float)


Número de grupos: 2
Número de terminales: 100
Número de terminales por grupo: 51
Ubicaciones de los grupos: [array([[35.75946832, 30.1381688 ],
       [27.24415915, 21.18273997],
       [32.29470565, 21.87936056],
       [44.58865004, 48.18313803],
       [19.17207594, 39.5862519 ],
       [26.44474599, 28.40222805],
       [46.27983191,  3.55180291],
       [ 4.35646499,  1.01091987],
       [41.63099228, 38.90783755],
       [43.50060741, 48.93091711],
       [39.95792821, 23.07396811],
       [39.02645881,  5.91372129],
       [31.99605107,  7.16766437],
       [47.23344585, 26.09241609],
       [20.733097  , 13.22778061],
       [38.71168447, 22.80751661],
       [28.42169744,  0.93949002],
       [30.88177485, 30.60478614],
       [30.84669984, 47.18740393],
       [34.09101496, 17.97539503],
       [21.85159769, 34.8815598 ],
       [ 3.01127358, 33.33833577],
       [33.53189348, 10.51912805],
       [ 6.44631488, 15.77141755],
       [18.18553855, 28.50983852],
       [21.93007

In [44]:
customer_profiles_table['terminales_disponibles']=customer_profiles_table.apply(lambda x : get_list_terminals_within_radius(x, x_y_terminals=x_y_terminals, r=50), axis=1)

In [45]:
transaction_table_customer_0=generate_transactions_table(customer_profiles_table.iloc[0], 
                                                         start_date = "2023-02-01", 
                                                         nb_days = 30)

In [46]:
transactions_df=customer_profiles_table.groupby('CLIENTE_ID').apply(lambda x : generate_transactions_table(x.iloc[0], nb_days=30)).reset_index(drop=True)

In [47]:
(customer_profiles_table, terminal_profiles_table, transactions_df)=\
    generate_dataset(n_customers = 1000, 
                     n_terminals = 100, 
                     nb_days=30, 
                     start_date="2023-02-01", 
                     r=5)

Generación tabla perfiles de cliente: 0.012s
Número de grupos: 3
Número de terminales: 100
Número de terminales por grupo: 34
Ubicaciones de los grupos: [array([[46.62786797,  6.4062224 ],
       [49.95202577, 11.80444885],
       [19.82903636, 19.39553706],
       [33.48730184, 46.77695354],
       [42.31554583, 15.66367585],
       [26.22740798, 22.17264469],
       [11.47886069, 26.72069545],
       [45.69810123, 22.8602404 ],
       [21.53492836, 46.95638947],
       [38.91946182, 35.7985258 ],
       [40.1378752 ,  4.64004043],
       [25.90762745, 43.2510126 ],
       [41.45734537, 41.48016797],
       [13.65249871,  2.96216007]]), array([[ 4.91734169, 21.05538125],
       [47.89447651, 26.65826425],
       [34.5938557 , 15.77578155],
       [34.32504638, 41.73128359],
       [ 0.91441387, 37.50721575],
       [49.44305445, 37.40828272],
       [14.0221996 , 39.46396642],
       [ 5.16130033, 22.39467631],
       [45.42977515, 14.68070742],
       [14.38876693,  6.50142861]]), ar

In [48]:
%time transactions_df = add_frauds(customer_profiles_table, terminal_profiles_table, transactions_df)
transactions_df.TX_FRAUD.mean()
transactions_df.TX_FRAUD.sum()

Number of frauds from scenario 1: 2832
Number of frauds from scenario 2: 2401
Number of frauds from scenario 3: 43
Number of frauds from scenario 4: 135
CPU times: user 216 ms, sys: 3.99 ms, total: 220 ms
Wall time: 219 ms


5411

### Exportación del dataset

In [33]:
DIR_OUTPUT = "../validation/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2023-02-01", "%Y-%m-%d")

for day in range(transactions_df.DIA_TX.max()+1):
    
    transactions_day = transactions_df[transactions_df.DIA_TX==day].sort_values('TIEMPO_DESDE_ULTIMA_TX_S')
    
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    # Protocol=4 required for Google Colab
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)